# N-Gram Language Models Implementation

In [2]:
import os


## Reading 20N dataset

In [11]:
def read_document_20N():
    """
    This method reads and writes the 20N Dataset
    :return:
    """
    new_file = os.path.join(os.getcwd(), 'datasets/20news_file')
    os.remove(new_file)

    folder_path = os.path.join(os.getcwd(), 'datasets/20news-18828')
    inner_dirs = os.listdir(folder_path)
    for dir in inner_dirs:
        if not dir.startswith('.'):
            #print(dir)
            dir_path = os.path.join(folder_path,dir)
            filenames = os.listdir(dir_path)
            for file in filenames:
                cur_path = os.path.join(dir_path,file)
                #print("Copying "+file)
                with open(cur_path,'r', errors="ignore") as firstfile, open(new_file,'a') as secondfile:
                    for line in firstfile:
                            secondfile.write(line)
    print("Archivo terminado")


read_document_20N()
#print(list(documentos.items())[0])


Archivo terminado


## Reading BAC dataset

In [ ]:
def read_document_BAC():
    """
    This method reads and writes the BAC Dataset
    :return:
    """
    new_file = os.path.join(os.getcwd(), 'datasets/bac_file')
    os.remove(new_file)

    folder_path = os.path.join(os.getcwd(), 'datasets/blogs')
    files = os.listdir(folder_path)
    for file in files:
        cur_path = os.path.join(folder_path,file)
            #print("Copying "+file)
            with open(cur_path,'r', errors="ignore") as firstfile, open(new_file,'a') as secondfile:
                for line in firstfile:
                        secondfile.write(line)
    print("Archivo terminado")


read_document_20N()